In [0]:
!pip install transformers

In [1]:
import json
import re
import pandas as pd


from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm
import torch
import numpy as np

import nltk
from nltk import tokenize
nltk.download('punkt')

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

idd = '1h_kUrWbKvZR0iRms8zKMUJO5zq9wbIXW' # (Cornell Newsroom Summarization Dataset)
downloaded_ = drive.CreateFile({'id':idd}) 
downloaded_.GetContentFile('train.jsonl')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
def clean_data(text):
    text = re.sub(r'\<[^>]*\>', '', text)
    text = re.sub(r'\\n', ' ', text)
    text = re.sub(r'&[a-z]{0,7};', ' ', text)
    text = re.sub(r'\s{2,10}', ' ', text)
    text = re.sub(r'\s{2,10}', ' ', text)
    text = re.sub(r"\\'", r"'", text)
    text = re.sub(r'\\x\d{1,4}', '', text)
    return text

def sent_first(text): # to gpt2
    return tokenize.sent_tokenize(text)[0]

def sent_splitter(text): # to classification model
    return ' '.join(tokenize.sent_tokenize(text)[:2])

In [0]:
# Загрузка файла и создание датафрейма
with open('train.jsonl', 'r') as json_file:
        json_list = list(json_file)
        
data = []
for json_str in json_list:
        result = json.loads(json_str)
        data.append(result['text'])

del json_list

df = pd.DataFrame()
df['text'] = data
df['label'] = ['real'] * len(data)
df['text'] = df['text'].apply(clean_data)

text_gpt2 = list(df['text'].apply(sent_first))
df.text  = df['text'].apply(sent_splitter)
text_real = list(df['text'])

In [9]:
MAX_LENGTH = 30
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [0]:
def generate_fake(texts, sampling_type, fake=[], tokenizer=tokenizer, model=model):
    """
    Функция для генерации гекстов в зависимости от типа сэмплирования. 
    sampling type: tuple (sampling_type : value)
    Записывает фейковые тексты в отдельный файл. (В DataGen_Class есть вариант еще и с обработкой real)
    Возвращает список фейковых текстов.
    """

    i = 0
    for ind, el in tqdm(enumerate(texts)):
        sent = generate_sentence(el, model, tokenizer, sampling_type)            
        sent = re.sub(r'\n', ' ', sent)
        fake.append(sent)

        if (i % 1000 == 0) and (i > 0):
            print('{} out of {} are not found in emb dict'.format(i, ind))
    return fake

def generate_sentence(sentence, model, tokenizer, sampling_type, max_length=MAX_LENGTH):    
    """
    Функция генерирует предложение в зависимости от типа сэмплирования
    """
    context = torch.tensor([tokenizer.encode(sentence)])
    max_length += context.size()[-1]
    
    if sampling_type[0] == 'beam_search':
        outputs = model.generate(input_ids=context, max_length=max_length,
                                 do_sample=True, num_beams=sampling_type[1])
    elif sampling_type[0] == 'temperature':
        outputs = model.generate(input_ids=context, max_length=max_length,
                                 do_sample=True, temperature=sampling_type[1])
    elif sampling_type[0] == 'top_k':
        outputs = model.generate(input_ids=context, max_length=max_length,
                                 do_sample=True, temperature=sampling_type[1])
    elif sampling_type[0] == 'nucleus':
        outputs = model.generate(input_ids=context, max_length=max_length,
                                 do_sample=True, top_p=sampling_type[1])
    else: # Иначе аргмакс
        outputs = model.generate(input_ids=context, max_length=max_length, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [0]:
texts_fake = []
texts_real = []
sampling_types = [('beam_search', 3), ('beam_search',5), ('temperature', 0.9), ('temperature', 0.8), ('top_k', 20), ('top_k', 100), ('nucleus', 0.9), ('nucleus', 0.8), ('argmax', 23)]


for ind, sampling_type in enumerate(sampling_types):
    generate_fake(text_real[ind*100000+1:(ind+1)*100000], sampling_type, fake=texts_fake, tokenizer=tokenizer,model=model)
    #texts_fake.extend(generate_fake(text_gpt2[ind*2+1:(ind+1)*2], sampling_type, fake=[], tokenizer=tokenizer,model=model))
    texts_real.extend(text_real[ind*2+1:(ind+1)*2])

In [51]:
texts_fake[:5], texts_real[:5]

(['WASHINGTON, Dec. 23 - The National Security Agency has traced and analyzed large volumes of telephone and Internet communications flowing into and out of the United States as part of the eavesdropping program that President Bush approved after the Sept. 11, 2001, attacks to hunt for evidence of terrorist activity, according to current and former government officials. Those surveillance programs, called "spies," now have a wide-ranging, sweeping reach, said the senior government officials and others who requested anonymity to',
  'BY A.J. ROTHSCHILD, M.D., is director of the Center for Healthy Health at the Massachusetts General Hospital.  She specializes in chronic',
  'All day, every day, Cheryl Bernstein thanks her 16-month-old son. She says he\'s a\'very happy puppy,\' and she\'s "trying to come up with a theme for him to explore," but she doesn',
  "Wednesday, April 19th 1995, 2:35AM Jail inmates flout the city's newest law every day, with no fear of punishment. They are held in

In [0]:
with open('fake.txt', 'w', encoding='utf-8') as f:    
    for item in texts_fake:
        f.write('{}\n'.format(item))

In [0]:
with open('real.txt'.format(sampling_type[0]), 'w', encoding='utf-8') as f:    
    for item in texts_real:
        f.write('{}\n'.format(item))